In [ ]:
import sys

import geopandas as gpd
import movingpandas as mpd
import pandas as pd
from movingpandas import Trajectory

from pymeos import pymeos_initialize, TGeogPointSeq, TGeogPointInst, TGeomPointInst, TGeomPointSeq, TPointSeq, TPointInst

pymeos_initialize()

plot_defaults = {'linewidth': 5, 'capstyle': 'round', 'figsize': (9, 3), 'legend': True}


In [ ]:
def create_point(row) -> TPointInst:
    return TGeogPointInst(string=f"{row['geometry']}@{row.name}")


def extract_wkt_from_traj(trajectory: Trajectory) -> str:
    res = "["
    for row in trajectory.df.iloc():
        res += f'{row.geometry.wkt}@{row.name},'
    res = res[:-1] + ']'
    return res


def create_sequence(trajectory: Trajectory) -> TPointSeq:
    wkt = extract_wkt_from_traj(trajectory)
    return TGeogPointSeq(string=wkt)

### Read AIS CSV

In [ ]:
pdf = pd.read_csv('data/aisinput.csv')
gdf = gpd.GeoDataFrame(pdf.drop(['latitude', 'longitude'], axis=1),
                       geometry=gpd.points_from_xy(pdf.longitude, pdf.latitude), crs=4326)
gdf.head()

In [ ]:
ships = mpd.TrajectoryCollection(gdf, 'mmsi', t='t')

In [ ]:
ship = ships.trajectories[1]
ship.df.head()

In [ ]:
ship.plot(column='speed', vmax=20, **plot_defaults)

In [ ]:
ship.df['MEOS Point'] = ship.df.apply(create_point, axis=1)
ship.df.head()

In [ ]:
ship.df

In [ ]:
ship.add_speed(overwrite=True)

In [ ]:
ship.add_distance(overwrite=True)
sum(ship.df[ship.distance_col_name])

In [ ]:
sequence = TGeogPointSeq(instant_list=list(ship.df['MEOS Point']), normalize=False)
sequence.length()

In [ ]:
sequence = TGeogPointSeq(instant_list=list(ship.df['MEOS Point']), normalize=True)
sequence.length()

In [ ]:
ship.plot(vmax=20, **plot_defaults)

In [ ]:
sequence.to_dataframe().plot(vmax=20, **plot_defaults)

In [ ]:
sequence.num_instants

In [ ]:
len(ship.df)

In [ ]:
sequence = create_sequence(ship)
sequence.length()

In [ ]:
ship.df['distance']